In [2]:
import re
import csv
from lxml import etree

In [3]:
inputFile = '../data/source/nb-allRecords.xml'
outputDir = '../data/xml/nb'
limit = 10
offset = 0

In [4]:
curatedDataFiles = [
    "../data/source/nb-curation-personen.csv",
    "../data/source/nb-curation-koerperschaften.csv",
    "../data/source/nb-curation-geografika.csv"
]
curatedKey = "Raw"
curatedFieldsToAdd = ["GND-Nummer", "GND-Kennung", "WD"]

In [16]:
curatedNamesFile = "../data/source/nb-curation-names.csv"

In [17]:
root = etree.parse(inputFile)

In [18]:
collection = root.getroot()

In [19]:
records = root.findall("Record")

In [20]:
curatedData = []
for curatedDataFile in curatedDataFiles:
    with open(curatedDataFile, 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            curatedData.append(row)

In [21]:
descriptors = root.xpath("//Descriptor")

In [22]:
for descriptor in descriptors:
    thesaurus = descriptor.find("Thesaurus").text
    key = descriptor.find("IdName").text
    try:
        dataToAdd = [d for d in curatedData if d['Thesaurus'] == thesaurus and d[curatedKey] == key][0]
    except:
        continue
    for field in curatedFieldsToAdd:
        if field in dataToAdd:
            el = etree.SubElement(descriptor, field)
            el.text = dataToAdd[field]

In [76]:
curatedNames = []
with open(curatedNamesFile, 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        curatedNames.append(row)

In [81]:
elementIdsWithCuratedNames = ['10817', '10927']
dataElementXPath = '|'.join(["DetailData/DataElement[@ElementId='%s']" % d for d in elementIdsWithCuratedNames])

for record in records:
    recordElements = record.xpath(dataElementXPath)
    recordDescriptors = record.xpath("Descriptors/Descriptor[Thesaurus/text()='Personen']")
    if len(recordElements):
        for recordElement in recordElements:
            values = recordElement.xpath("ElementValue")
            for value in values:
                name = value.find("TextValue").text
                print(name)
                for descriptor in recordDescriptors:
                    idName = descriptor.find("IdName").text
                    print(idName)
    print("--")

Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max

Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max

Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max

Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Berchem, Max van
FotografIn  (Personen\B\Berchem, Max van (1863 - 1921))
--
Spelterini, Eduard
FotografIn  (Personen\S\Spelterini, Eduard (1852 - 1931))
--
Spelterini, Eduard
FotografIn  (Personen\S\Spelterini, Eduard (1852 - 1931))
--
Spelterini, Eduard
FotografIn  (Personen\S\Spelterini, Eduard (1852 - 1931))
--
Spelterini, Eduard
FotografIn  (Personen\S\Spelterini, Eduard (1852 - 1931))
--
Spelterini, Eduard
FotografIn  (Personen\S\Spelterini, Eduard (1852 - 1931))
--
Spelterini, Eduard
FotografIn  (Personen\S\Spelterini, Eduard (1852 

Brun, Louis Auguste
BildendeR KünstlerIn  (Personen\B\Brun, Louis Auguste (1758 - 1815))
--
Burgdorfer, Daniel David
BildendeR KünstlerIn  (Personen\B\Burgdorfer, Daniel David (1800 - 1861))
--
Burgdorfer, Daniel David
BildendeR KünstlerIn  (Personen\B\Burgdorfer, Daniel David (1800 - 1861))
--
Burgdorfer, Daniel David
BildendeR KünstlerIn  (Personen\B\Burgdorfer, Daniel David (1800 - 1861))
--
Burri, Johann Ulrich [MalerIn/ZeichnerIn]
BildendeR KünstlerIn  (Personen\S\Sperli, Johann Jakob (1794 - 1843))
BildendeR KünstlerIn  (Personen\B\Burri, Johann Ulrich (1802 - 1870))
Vertrieb/Verlag  (Personen\B\Burri, Johann Ulrich (1802 - 1870))
Sperli, Johann Jakob [StecherIn]
BildendeR KünstlerIn  (Personen\S\Sperli, Johann Jakob (1794 - 1843))
BildendeR KünstlerIn  (Personen\B\Burri, Johann Ulrich (1802 - 1870))
Vertrieb/Verlag  (Personen\B\Burri, Johann Ulrich (1802 - 1870))
--
Büchel, Emanuel
BildendeR KünstlerIn  (Personen\B\Büchel, Emanuel (1705 - 1775))
--
Büchel, Emanuel
BildendeR Küns

...

Weibel, Jakob Samuel
BildendeR KünstlerIn  (Personen\W\Weibel, Jakob Samuel (1771 - 1846))
--
König, Franz Niklaus
BildendeR KünstlerIn  (Personen\K\König, Franz Niklaus (1765 - 1832))
--
[Huber, Rudolf?]
BildendeR KünstlerIn  (Personen\H\Huber, Rudolf (1770 - 1844))
--
Locher [MalerIn/ZeichnerIn]
Vertrieb/Verlag  (Personen\L\Lamy, Johann Peter (ca. 1791 - ca. 1838))
--
Weibel, Jakob Samuel [MalerIn/ZeichnerIn]
BildendeR KünstlerIn  (Personen\W\Weibel, Jakob Samuel (1771 - 1846))
--
Hürlimann, Johann [StecherIn]
BildendeR KünstlerIn  (Personen\H\Hürlimann, Johann (1793 - 1850))
BildendeR KünstlerIn  (Personen\L\Lory, Gabriel (1784 - 1846))
Lory, Gabriel "Fils" [MalerIn/ZeichnerIn]
BildendeR KünstlerIn  (Personen\H\Hürlimann, Johann (1793 - 1850))
BildendeR KünstlerIn  (Personen\L\Lory, Gabriel (1784 - 1846))
--
Lory, Gabriel "Fils"
BildendeR KünstlerIn  (Personen\L\Lory, Gabriel (1784 - 1846))
--
Meyer, Johann Jakob
BildendeR KünstlerIn  (Personen\M\Meyer, Johann Jakob (1787 - 1858))
-

Jenny, [Arnold?] [peintre/dessinateur]
BildendeR KünstlerIn  (Personen\H\Himely, Sigismond (1801 - 1872))
Vertrieb/Verlag  (Personen\B\Bleuler, Johann Ludwig (1792 - 1850))
BildendeR KünstlerIn  (Personen\J\Jenny, Arnold (1831 - 1881))
Himely, Sigismond [graveur]
BildendeR KünstlerIn  (Personen\H\Himely, Sigismond (1801 - 1872))
Vertrieb/Verlag  (Personen\B\Bleuler, Johann Ludwig (1792 - 1850))
BildendeR KünstlerIn  (Personen\J\Jenny, Arnold (1831 - 1881))
--
--
Schmid, David Alois [peintre/dessinteur]
BildendeR KünstlerIn  (Personen\B\Burkhard, Kaspar (1810 - 1882))
BildendeR KünstlerIn  (Personen\S\Schmid, David Alois (1791 - 1861))
Burkhard, Kaspar [graveur]
BildendeR KünstlerIn  (Personen\B\Burkhard, Kaspar (1810 - 1882))
BildendeR KünstlerIn  (Personen\S\Schmid, David Alois (1791 - 1861))
--
Sabatier, Léon Jean-Baptiste
BildendeR KünstlerIn  (Personen\S\Sabatier, Léon Jean-Baptiste ( - 1887))
Vertrieb/Verlag  (Personen\S\Sinnett, F. (19. Jh.))
--
Albanis Beaumont, Jean-François
Bi

Walliser [MalerIn/ZeichnerIn]
BildendeR KünstlerIn  (Personen\S\Siegfried, Heinrich (1814 - 1889))
Vertrieb/Verlag  (Personen\D\Dikenmann, Johann Rudolf (1793 - 1883))
Siegfried, Heinrich [StecherIn]
BildendeR KünstlerIn  (Personen\S\Siegfried, Heinrich (1814 - 1889))
Vertrieb/Verlag  (Personen\D\Dikenmann, Johann Rudolf (1793 - 1883))
--
Weber, Lukas [StecherIn]
BildendeR KünstlerIn  (Personen\O\Oppermann, Wilhelm Ulrich (1786 - 1852))
BildendeR KünstlerIn  (Personen\W\Weber, Lukas (1811 - 1860))
Vertrieb/Verlag  (Personen\L\Leuthold, Hans Felix (19. Jh.))
Oppermann, Wilhelm Ulrich [MalerIn/ZeichnerIn]
BildendeR KünstlerIn  (Personen\O\Oppermann, Wilhelm Ulrich (1786 - 1852))
BildendeR KünstlerIn  (Personen\W\Weber, Lukas (1811 - 1860))
Vertrieb/Verlag  (Personen\L\Leuthold, Hans Felix (19. Jh.))
--
Weber, Lukas [StecherIn]
BildendeR KünstlerIn  (Personen\O\Oppermann, Wilhelm Ulrich (1786 - 1852))
BildendeR KünstlerIn  (Personen\W\Weber, Lukas (1811 - 1860))
Vertrieb/Verlag  (Personen

Winterlin, Anton [MalerIn/ZeichnerIn]
BildendeR KünstlerIn  (Personen\W\Winterlin, Anton (1805 - 1894))
BildendeR KünstlerIn  (Personen\B\Beck, Jakob Theophil (1814 - 1903))
Beck, Jakob Theophil [StecherIn]
BildendeR KünstlerIn  (Personen\W\Winterlin, Anton (1805 - 1894))
BildendeR KünstlerIn  (Personen\B\Beck, Jakob Theophil (1814 - 1903))
--
--
--
Vallotton, Félix
BildendeR KünstlerIn  (Personen\V\Vallotton, Félix (1865 - 1925))
--
Luttringhausen, Johann Heinrich [MalerIn/ZeichnerIn]
BildendeR KünstlerIn  (Personen\F\Falkeisen, Johann Jakob (1804 - 1883))
BildendeR KünstlerIn  (Personen\L\Luttringhausen, Johann Heinrich (1783 - 1857))
Falkeisen, Johann Jakob [StecherIn]
BildendeR KünstlerIn  (Personen\F\Falkeisen, Johann Jakob (1804 - 1883))
BildendeR KünstlerIn  (Personen\L\Luttringhausen, Johann Heinrich (1783 - 1857))
--
Lory, Gabriel "Fils" [MalerIn/ZeichnerIn]
BildendeR KünstlerIn  (Personen\L\Lory, Gabriel (1784 - 1846))
--
--
Zurkirchen, Joseph Bartholomäus [MalerIn/ZeichnerIn

Grundmann, [Samuel] [peintre/dessinateur]
Vertrieb/Verlag  (Personen\L\Lamy, Johann Peter (ca. 1791 - ca. 1838))
--
Lory, Gabriel "Fils" [peintre/dessinateur]
BildendeR KünstlerIn  (Personen\L\Lory, Gabriel (1784 - 1846))
BildendeR KünstlerIn  (Personen\H\Himely, Sigismond (1801 - 1872))
Vertrieb/Verlag  (Personen\O\Osterwald, Jean Frédéric d' (1773 - 1850))
Himely, Sigismond [graveur]
BildendeR KünstlerIn  (Personen\L\Lory, Gabriel (1784 - 1846))
BildendeR KünstlerIn  (Personen\H\Himely, Sigismond (1801 - 1872))
Vertrieb/Verlag  (Personen\O\Osterwald, Jean Frédéric d' (1773 - 1850))
--
Grundmann, [Samuel] [peintre/dessinateur]
Vertrieb/Verlag  (Personen\L\Lamy, Johann Peter (ca. 1791 - ca. 1838))
--
Winterlin, Anton [peintre/dessinateur]
BildendeR KünstlerIn  (Personen\W\Winterlin, Anton (1805 - 1894))
BildendeR KünstlerIn  (Personen\B\Beck, Jakob Theophil (1814 - 1903))
Beck, Jakob Theophil [graveur]
BildendeR KünstlerIn  (Personen\W\Winterlin, Anton (1805 - 1894))
BildendeR Künstler

In [24]:
# elementIdsToAddDescriptors = ['10817', '10927']
# xpath = '|'.join(["//DataElement[@ElementId='%s']" % d for d in elementIdsToAddDescriptors])
# elementsToAddDescriptors = root.xpath(xpath)

In [47]:
# descriptorIndex = {}
# descriptorKeys = []
# for descriptor in descriptors:
#     idName = descriptor.find("IdName").text
#     if idName not in descriptorIndex:
#         descriptorIndex[idName] = descriptor
#         descriptorKeys.append(idName)

In [60]:
# unmatchedNames = []
# for element in elementsToAddDescriptors:
#     values = element.xpath("ElementValue")
#     for value in values:
#         name = value.find("TextValue").text
        
#         try:
#             matchedCuratedName = [d for d in curatedNames if d['Name'] in name][0]
#         except:
#             print("Could not find %s in curated names" % name)
            
#         try:
#             matchedDescriptorKey = [d for d in descriptorKeys if matchedCuratedName['Name'] in d][0]
#         except:
#             unmatchedNames.append(matchedCuratedName['Name'])

# unmatchedNames = list(set(unmatchedNames))
# unmatchedNames.sort()

In [10]:
def getDateForDateElement(date):
    if not date.text:
        return False
        
    patternCeYear = r'\+\d{4}'
    if re.match(patternCeYear, date.text):
        year = date.text[1:].zfill(4)
        if date.tag == 'FromDate':
            return "%s-01-01" % year
        else:
            return "%s-12-31" % year
    return False

In [11]:
dates = root.xpath("//FromDate|//ToDate")
for date in dates:
    fullDate = getDateForDateElement(date)
    if fullDate:
        date.set("fullDate", fullDate)

In [12]:
for record in records[offset:limit+offset]:
    collection.clear()
    recordId = record.get("Id")
        
    collection.append(record)
    outputFile = "%s/nb-record-%s.xml" % (outputDir, recordId)
    with open(outputFile, 'wb') as f:
        f.write(etree.tostring(collection, xml_declaration=True, pretty_print=True, encoding="UTF-8"))